In [35]:
import os
import sys
import pandas as pd

current_dir = os.getcwd()  # Obtiene el directorio de trabajo actual
pyfcs_dir = os.path.abspath(os.path.join(current_dir, '..', '..'))

# Add the PyFCS path to sys.path
sys.path.append(pyfcs_dir)

# Caso 1. Software Validación Humana

Genero diccionario con el excel que contiene los resultados.

In [36]:
# Ruta del archivo de Excel
file_path = os.path.join(os.getcwd(), "Results", "Val_Results.xlsx")

# Cargar todas las hojas en un diccionario
sheets_dict = pd.read_excel(file_path, sheet_name=None)

# Convertir cada hoja en una matriz y almacenarla en un diccionario
data_dict = {sheet_name: df for sheet_name, df in sheets_dict.items()}

# Mostrar los nombres de las hojas cargadas
print(f"Hojas cargadas: {list(data_dict.keys())}")


Hojas cargadas: ['MariaTejada']


Procesado de los resultados

In [60]:
def clean_values_confidence(values, confidence):
    values_list = [v.strip() for v in str(values).split(",")]
    confidence_list = [float(c) for c in str(confidence).split(",")]

    # Asegurar que tengan el mismo tamaño agregando vacíos si falta alguno
    while len(values_list) < len(confidence_list):
        values_list.append("")
    while len(confidence_list) < len(values_list):
        confidence_list.append(0.0)

    # Filtrar solo las posiciones donde Value NO está vacío y Confidence > 0
    cleaned_pairs = [(v, c) for v, c in zip(values_list, confidence_list) if v and c > 0]

    # Separar en listas nuevamente
    cleaned_values = [v for v, c in cleaned_pairs]
    cleaned_confidence = [c for v, c in cleaned_pairs]

    return ", ".join(cleaned_values), ", ".join(map(str, cleaned_confidence))



# Diccionario donde guardaremos los datos procesados
dent_data = {}
for sheet_name, df in data_dict.items():
    new_rows = []

    for _, row in df.iterrows():  # Iterar sobre cada fila
        tooth = row['Tooth']  # Mantener la columna 'Tooth'

        # Procesar todas las combinaciones de Value/Confidence
        upper_value, upper_confidence = clean_values_confidence(row['Upper Value'], row['Upper Confidence'])
        central_value, central_confidence = clean_values_confidence(row['Central Value'], row['Central Confidence'])
        lower_value, lower_confidence = clean_values_confidence(row['Lower Value'], row['Lower Confidence'])

        # Crear nueva fila con datos procesados
        new_rows.append([tooth, upper_value, upper_confidence, central_value, central_confidence, lower_value, lower_confidence])

    # Convertir la lista de filas procesadas a un DataFrame
    processed_df = pd.DataFrame(new_rows, columns=[
        'Tooth', 'Upper Value', 'Upper Confidence', 'Central Value', 'Central Confidence', 'Lower Value', 'Lower Confidence'
    ])
    dent_data[sheet_name] = processed_df

    print(f"Procesada hoja: {sheet_name}")



Procesada hoja: MariaTejada


In [61]:
print(clean_values_confidence("A1, , B2", "0.9, 0.5, 0.2"))
print(clean_values_confidence(" , C3, ", "0.7, 0.6, 0.0"))

('A1, B2', '0.9, 0.2')
('C3', '0.6')


In [62]:
for sheet_name, df in dent_data.items():
    print(f"\nHoja: {sheet_name}")
    print(df.head().to_string(index=False))  



Hoja: MariaTejada
Tooth Upper Value Upper Confidence Central Value Central Confidence Lower Value Lower Confidence
   A1      C1, D2         0.3, 0.2        A1, B1           0.9, 0.1      C1, B1         0.2, 0.1
   A2      C3, C2         0.5, 0.2            A2                0.9          D2              0.3
   A3          D4              0.5            A3                0.9          C2              0.5
 A3_5          A4              0.2          A3_5                0.7          A4              0.2
   A4          C4              0.2            A4                0.9          C4              0.4


## Carga de ambos resultados de PyFCS

In [64]:
import os
import pandas as pd

# Función para analizar los valores
def parse_values(cell):
    if isinstance(cell, str):
        return eval(cell)  # Convierte la cadena en una lista de tuplas
    return []

# Función para dividir los valores por columna
def split_values(df, col):
    return df[col].str.split(', ').apply(lambda x: x)

# Leer el archivo Excel
file_path = os.path.join(os.getcwd(), "Results", "PyFCS", "results_opt_1.xlsx")
df = pd.read_excel(file_path)

# Eliminar la extensión .png de la columna Imagen y renombrarla a Tooth
df = df.rename(columns={"image": "Tooth"})
df["Tooth"] = df["Tooth"].str.replace(".png", "", regex=False)

# Aplicar la función parse_values a las columnas 'top', 'middle', 'bottom'
df["top"] = df["top"].apply(parse_values)
df["middle"] = df["middle"].apply(parse_values)
df["bottom"] = df["bottom"].apply(parse_values)

# Crear nuevas columnas para el formato requerido
df["Upper Value"] = df["top"].apply(lambda x: ", ".join([i[0] for i in x]))
df["Upper Confidence"] = df["top"].apply(lambda x: ", ".join([str(i[1]) for i in x]))
df["Central Value"] = df["middle"].apply(lambda x: ", ".join([i[0] for i in x]))
df["Central Confidence"] = df["middle"].apply(lambda x: ", ".join([str(i[1]) for i in x]))
df["Lower Value"] = df["bottom"].apply(lambda x: ", ".join([i[0] for i in x]))
df["Lower Confidence"] = df["bottom"].apply(lambda x: ", ".join([str(i[1]) for i in x]))

# Eliminar las columnas 'top', 'middle', 'bottom'
df = df.drop(columns=["top", "middle", "bottom"])

# Ahora, tenemos la estructura que buscas.
# Solo necesitamos el DataFrame resultante con las columnas organizadas.
pyfcs_opt_1 = df[['Tooth', 'Upper Value', 'Upper Confidence', 'Central Value', 
                'Central Confidence', 'Lower Value', 'Lower Confidence']]

# Mostrar el DataFrame resultante
print(pyfcs_opt_1.head().to_string(index=False))  

Tooth Upper Value    Upper Confidence Central Value  Central Confidence  Lower Value    Lower Confidence
   A1  A1, D2, C1  0.417, 0.334, 0.15    A1, B1, D2       1.0, 0.0, 0.0   D2, A1, C4 0.475, 0.306, 0.096
   A2  C3, C4, A3 0.219, 0.205, 0.179    A2, B2, A3 0.666, 0.319, 0.008   B2, D3, C4   0.4, 0.296, 0.148
   A3  D4, C3, C4 0.331, 0.285, 0.229    A3, B2, D4  0.83, 0.103, 0.049   D4, C2, C4 0.282, 0.282, 0.227
 A3_5  B4, C4, A4 0.435, 0.365, 0.097  A3.5, B3, B4 0.421, 0.322, 0.245 C4, A3.5, B4 0.338, 0.272, 0.193
   A4  C4, A4, B4 0.546, 0.279, 0.101  A4, C3, A3.5 0.711, 0.225, 0.062   C4, A4, C3 0.718, 0.254, 0.018


{'A1': {'Upper Value': 'A1, D2, C1',
  'Upper Confidence': '0.417, 0.334, 0.15',
  'Central Value': 'A1, B1, D2',
  'Central Confidence': '1.0, 0.0, 0.0',
  'Lower Value': 'D2, A1, C4',
  'Lower Confidence': '0.475, 0.306, 0.096'},
 'A2': {'Upper Value': 'C3, C4, A3',
  'Upper Confidence': '0.219, 0.205, 0.179',
  'Central Value': 'A2, B2, A3',
  'Central Confidence': '0.666, 0.319, 0.008',
  'Lower Value': 'B2, D3, C4',
  'Lower Confidence': '0.4, 0.296, 0.148'},
 'A3': {'Upper Value': 'D4, C3, C4',
  'Upper Confidence': '0.331, 0.285, 0.229',
  'Central Value': 'A3, B2, D4',
  'Central Confidence': '0.83, 0.103, 0.049',
  'Lower Value': 'D4, C2, C4',
  'Lower Confidence': '0.282, 0.282, 0.227'},
 'A3_5': {'Upper Value': 'B4, C4, A4',
  'Upper Confidence': '0.435, 0.365, 0.097',
  'Central Value': 'A3.5, B3, B4',
  'Central Confidence': '0.421, 0.322, 0.245',
  'Lower Value': 'C4, A3.5, B4',
  'Lower Confidence': '0.338, 0.272, 0.193'},
 'A4': {'Upper Value': 'C4, A4, B4',
  'Upper Co